# Rock Paper Scissors - AI
## Model training

### Imports

In [2]:
#lib imports
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#local imports
from utils import *
from plotting import *
from model_creator import *

print('TensorFlow version: ', tf.__version__)
print('Keras version: ', keras.__version__)
print('Python version: ', os.sys.version)

TensorFlow version:  2.16.1
Keras version:  3.1.1
Python version:  3.12.2 | packaged by conda-forge | (main, Feb 16 2024, 20:42:31) [MSC v.1937 64 bit (AMD64)]


### Dataset preprocessing

In [3]:
local_dir = './'
original_data_dir = os.path.join(local_dir, 'original_data')
dataset_dir = os.path.join(local_dir, 'dataset')
models_dir = os.path.join(local_dir, 'models')

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')
test_dir = os.path.join(dataset_dir, 'test')

if not os.path.exists(original_data_dir):
    raise FileNotFoundError('Original data directory not found')

classes = ['rock', 'paper', 'scissors']

if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
    split_dataset(classes, original_data_dir, dataset_dir)

for path in [train_dir, val_dir, test_dir]:
    if not os.path.exists(path):
        raise FileNotFoundError(f'{path} not found')
    
    print(path)
    for class_name in classes:
        print(f'    - {class_name}: {len(os.listdir(os.path.join(path, class_name)))} images')
  

./dataset\train
    - rock: 580 images
    - paper: 569 images
    - scissors: 600 images
./dataset\val
    - rock: 73 images
    - paper: 71 images
    - scissors: 75 images
./dataset\test
    - rock: 73 images
    - paper: 72 images
    - scissors: 75 images


In [4]:
check_for_duplicates_in_dataset('./dataset')

No duplicates found in dataset
2188 files found in dataset


### Data augmentation

In [5]:
# Image target size
img_rows, img_cols = 224, 224

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

test_datagen = ImageDataGenerator(rescale=1./255)

# Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_rows, img_cols),
    batch_size=32,
    class_mode='categorical')

val_genetator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(img_rows, img_cols),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_rows, img_cols),
    batch_size=32,
    class_mode='categorical')


Found 1749 images belonging to 3 classes.
Found 219 images belonging to 3 classes.
Found 220 images belonging to 3 classes.


### Model CNN

In [6]:
model_name = 'simple_cnn'
load_model = False

model = create_simple_cnn_model(input_shape=(224, 224, 3), num_classes=3)

if load_model:
    assert os.path.exists(model_path), 'Model not found'
    model = tf.keras.models.load_model(model_path)
    print(f'Model loaded: {model_path}')

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])
model.summary()

c:\Users\adria\anaconda3\envs\ML\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    11,944,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,963,587 (45.64 MB)

 Trainable params: 11,963,587 (45.64 MB)

 Non-trainable params: 0 (0.00 B)

### Training

In [ ]:
epochs = 100
save_model = True

hist = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples // train_generator.batch_size,
      epochs=epochs,
      validation_data=val_genetator,
      validation_steps=val_genetator.samples // val_genetator.batch_size)

if save_model:
    save_model(model, model_name, models_dir, history=hist.history)

Epoch 1/100


c:\Users\adria\anaconda3\envs\ML\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


54/54 ━━━━━━━━━━━━━━━━━━━━ 35s 579ms/step - acc: 0.3938 - loss: 1.0995 - val_acc: 0.3958 - val_loss: 1.0113
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.3750 - loss: 0.5333 - val_acc: 0.4815 - val_loss: 0.4742
Epoch 3/100


c:\Users\adria\anaconda3\envs\ML\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


54/54 ━━━━━━━━━━━━━━━━━━━━ 23s 401ms/step - acc: 0.4714 - loss: 1.0246 - val_acc: 0.5677 - val_loss: 0.9339
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 936us/step - acc: 0.5938 - loss: 0.5219 - val_acc: 0.5926 - val_loss: 0.4645
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 22s 385ms/step - acc: 0.5288 - loss: 0.9755 - val_acc: 0.7500 - val_loss: 0.8529
Epoch 6/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.6250 - loss: 0.4887 - val_acc: 0.7407 - val_loss: 0.4189
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 22s 388ms/step - acc: 0.5560 - loss: 0.9297 - val_acc: 0.7917 - val_loss: 0.7810
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.4375 - loss: 0.4612 - val_acc: 0.7778 - val_loss: 0.4277
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 24s 413ms/step - acc: 0.6193 - loss: 0.8767 - val_acc: 0.6875 - val_loss: 0.7401
Epoch 10/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.6562 - loss: 0.3451 - val_acc: 0.5556 - val_loss: 0.4519
Epoch 11/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 24s 424ms/step - acc:

In [ ]:
plot_accuracy_and_loss(hist.history)